In [87]:
import os
import pandas as pd

#Read in Liberal data from txt files
data = []

for f in os.listdir('./libdata'):
    with open('./libdata/' + f, 'r') as myfile:
        data.append(myfile.read())
        
srsLib = pd.Series(data, name = 'text')

#Read in Conservative data from txt files
data = []

for f in os.listdir('./condata'):
    with open('./condata/' + f, 'r') as myfile:
        data.append(myfile.read())

srsCon = pd.Series(data, name = 'text')

#len('Message-ID: <2262872.1075863600393.JavaMail.evans@thyme>')
#dfSpam = pd.concat([enron, pd.Series([0]*len(enron), name = 'isFood')], axis = 1)
#dfSpam.to_csv('enron.csv')
#dfSpam.head()

print (srsLib, srsCon)

0    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
1    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
2    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
3    SECTIONS HOME SEARCHSKIP TO CONTENTSKIP TO NAV...
Name: text, dtype: object 0    DOW JONES, A NEWS CORP COMPANY \nDJIA\n?\n1988...
1    \n \n       \nDailyCaller\n\n \n\nHOME\nPOLITI...
2    \n \n       \nDailyCaller\n\n \n\nHOME\nPOLITI...
3    \nPowered by Tynt\nshare on Facebooktweet this...
Name: text, dtype: object


In [88]:
#preprocessing text, split string into list of words
escapes = [chr(char) for char in range(1,32)]
escapes.append('\x92')
escapes.append('\x97')

#remove escape chars as spaces

from nltk.tokenize import RegexpTokenizer



def preProc(text):
    return ' '.join(
        RegexpTokenizer(r'\w+').tokenize(
        str.lower(
            text
            )
        )
    )

#    Stem the tokens

#Join and classify as lib or not lib
dfLib = pd.concat([srsLib.apply(preProc), pd.Series([1]*len(srsLib), name = 'isLib')], axis = 1)
dfCon = pd.concat([srsCon.apply(preProc), pd.Series([0]*len(srsCon), name = 'isLib')], axis = 1)
dfData = dfLib.append(dfCon)
dfData.head()

,text,isLib
0,sections home searchskip to contentskip to nav...,1
1,sections home searchskip to contentskip to nav...,1
2,sections home searchskip to contentskip to nav...,1
3,sections home searchskip to contentskip to nav...,1
0,dow jones a news corp company djia 19885 73 0 ...,0


In [89]:
#Split into train and test set
import numpy as np
import sklearn
from sklearn.cross_validation import train_test_split

X = dfData['text']
y = dfData['isLib']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [82]:
#Create vector of word counts SKIP
#from sklearn.feature_extraction.text import CountVectorizer
#count_vect = CountVectorizer()

#X_train_counts = count_vect.fit_transform(X_train)

In [83]:
#Create tfidf sprase matrix SKIP
#from sklearn.feature_extraction.text import TfidfTransformer
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [84]:
#fit model SKIP
#from sklearn.naive_bayes import MultinomialNB

#clf = MultinomialNB().fit(X_train_tfidf, y_train) 

In [90]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

In [91]:
text_clf.fit(X_train, y_train)

from sklearn.externals import joblib
joblib.dump(text_clf, 'text_clf.pkl')

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.33333333333333331